## Deforestation Prediction Models - Matale District

Advanced machine learning models for forecasting deforestation trends using cloud-masked Landsat data.

### 🎯 **Models Implemented**
1. **Random Forest Regression** - Ensemble learning with temporal features
2. **Random Forest Pixel-Level** - Spatial feature analysis (simulated)
3. **XGBoost Regression** - Gradient boosting for improved accuracy
4. **SARIMA** - Seasonal Autoregressive Integrated Moving Average

### 📊 **Analysis Pipeline**
- Data loading & cloud coverage integration
- Quality filtering (>50% cloud removal)
- Feature engineering (temporal + spatial)
- Model training & evaluation
- Comparative performance analysis
- Long-term forecasting (2026-2030)

### 🔑 **Key Features**
- Cloud-masked data for accurate baseline
- Multi-model ensemble approach
- Temporal and spatial feature extraction
- Comprehensive performance metrics (MAE, RMSE, R²)
- Future projections with cumulative impact analysis

In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.cluster import KMeans
from xgboost import XGBRegressor
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings("ignore")

print("="*70)
print("DEFORESTATION PREDICTION MODELS - MATALE DISTRICT")
print("Multi-Model Comparison: RF, RF-Pixel, XGBoost, SARIMA")
print("="*70)

# ============================================================
# 1. CONFIGURATION & SETUP
# ============================================================
BASE_DIR = r"D:\Satellite Image Processing\Deforestation_Matale"
MONTHLY_FILE = os.path.join(BASE_DIR, "Processed_Monthly", "Monthly_Deforestation_Stats.csv")
CLOUD_FILE = os.path.join(BASE_DIR, "Processed_Monthly", "Cloud_Coverage_Stats.csv")
OUTPUT_DIR = os.path.join(BASE_DIR, "Model_Comparison")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ============================================================
# 2. DATA LOADING & CLOUD COVERAGE INTEGRATION
# ============================================================
print("\n" + "="*70)
print("2. DATA LOADING & CLOUD COVERAGE INTEGRATION")
print("="*70)

def load_deforestation_data():
    """Load deforestation data with cloud coverage metrics"""
    if not os.path.exists(MONTHLY_FILE):
        raise FileNotFoundError(f"Data file not found: {MONTHLY_FILE}")

    df = pd.read_csv(MONTHLY_FILE, parse_dates=["Date"]).sort_values("Date")
    print(f"✓ Loaded monthly dataset: {len(df)} rows")
    print(f"  Date range: {df['Date'].min().date()} → {df['Date'].max().date()}")

    # Load cloud coverage if available
    if os.path.exists(CLOUD_FILE):
        cloud_df = pd.read_csv(CLOUD_FILE, parse_dates=["date"]).rename(columns={"date": "Date"})
        df = df.merge(cloud_df[["Date", "cloud_coverage"]], on="Date", how="left")
        df.rename(columns={"cloud_coverage": "Cloud_Coverage"}, inplace=True)

        if "Cloud_Coverage" in df.columns:
            avg_cloud = df['Cloud_Coverage'].mean()
            print(f"✓ Cloud coverage integrated")
            print(f"  Average cloud coverage: {avg_cloud:.1f}%")
    else:
        print("⚠ Cloud coverage file not found (older data format)")

    return df

df = load_deforestation_data()

# ============================================================
# 3. DATA PREPROCESSING & QUALITY FILTERING
# ============================================================
print("\n" + "="*70)
print("3. DATA PREPROCESSING & QUALITY FILTERING")
print("="*70)

# Remove high cloud coverage scenes
original_len = len(df)
if "Cloud_Coverage" in df.columns:
    df = df[df["Cloud_Coverage"] <= 50].copy()
    removed = original_len - len(df)
    if removed > 0:
        print(f"✓ Removed {removed} scenes with >50% cloud coverage")

# Handle missing values
if df["Deforested_Area_km2"].isna().any():
    missing_count = df["Deforested_Area_km2"].isna().sum()
    df["Deforested_Area_km2"].interpolate(method='linear', inplace=True)
    print(f"✓ Interpolated {missing_count} missing deforestation values")

print(f"✓ Final dataset size: {len(df)} scenes")

# Train/Test split
train_size = int(len(df) * 0.85)
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]

y_train, y_test = train_df["Deforested_Area_km2"], test_df["Deforested_Area_km2"]
X_train_index, X_test_index = train_df["Date"], test_df["Date"]

print(f"✓ Train/Test split: {len(train_df)} / {len(test_df)} scenes")
print(f"  Training period: {train_df['Date'].min().date()} → {train_df['Date'].max().date()}")
print(f"  Testing period: {test_df['Date'].min().date()} → {test_df['Date'].max().date()}")

# ============================================================
# 4. FEATURE ENGINEERING
# ============================================================
print("\n" + "="*70)
print("4. FEATURE ENGINEERING")
print("="*70)

# Temporal features
df["Month"] = df["Date"].dt.month
df["Year"] = df["Date"].dt.year
df["Quarter"] = df["Date"].dt.quarter
df["DayOfYear"] = df["Date"].dt.dayofyear

# Base feature set
feature_cols = ["Month", "Year", "Quarter"]

# Add cloud coverage if available
if "Cloud_Coverage" in df.columns:
    feature_cols.append("Cloud_Coverage")
    print("✓ Added cloud coverage as feature")

# Add valid pixels if available
if "Valid_Pixels" in df.columns:
    feature_cols.append("Valid_Pixels")
    print("✓ Added valid pixel count as feature")

print(f"✓ Feature set: {feature_cols}")

# Prepare feature matrices
X = df[feature_cols]
y = df["Deforested_Area_km2"]

X_train = X.iloc[:train_size]
X_test = X.iloc[train_size:]
y_train_ml = y.iloc[:train_size]
y_test_ml = y.iloc[train_size:]

# Simulated pixel-level features (placeholder for future spatial analysis)
print("\n✓ Generating simulated pixel-level features...")
np.random.seed(42)
df["Pixel_Mean"] = np.random.uniform(0, 1, len(df))
df["Pixel_Variance"] = np.random.uniform(0, 0.3, len(df))
df["Pixel_Texture"] = np.random.uniform(0, 0.1, len(df))

px_cols = ["Pixel_Mean", "Pixel_Variance", "Pixel_Texture"]

X_px = df[px_cols]
X_px_train = X_px.iloc[:train_size]
X_px_test = X_px.iloc[train_size:]

# ============================================================
# 5. MODEL TRAINING & EVALUATION
# ============================================================

# -----------------------------------------------------------
# 5.1 Random Forest Regression
# -----------------------------------------------------------
print("\n" + "="*70)
print("5.1 RANDOM FOREST REGRESSION")
print("="*70)

rf = RandomForestRegressor(n_estimators=400, max_depth=12, random_state=42)
rf.fit(X_train, y_train_ml)
rf_pred = rf.predict(X_test)

rf_mae = mean_absolute_error(y_test_ml, rf_pred)
rf_rmse = np.sqrt(mean_squared_error(y_test_ml, rf_pred))
rf_r2 = r2_score(y_test_ml, rf_pred)

print(f"✓ Random Forest Performance:")
print(f"  MAE:  {rf_mae:.3f} km²")
print(f"  RMSE: {rf_rmse:.3f} km²")
print(f"  R²:   {rf_r2:.3f}")

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)
print(f"\n  Feature Importance:")
for _, row in feature_importance.iterrows():
    print(f"    {row['Feature']}: {row['Importance']:.3f}")

# -----------------------------------------------------------
# 5.2 Random Forest Pixel-Level Model (Simulated)
# -----------------------------------------------------------
print("\n" + "="*70)
print("5.2 RANDOM FOREST - PIXEL LEVEL (Simulated)")
print("="*70)
print("Note: Using simulated pixel features as placeholder")

rf_px = RandomForestRegressor(n_estimators=300, max_depth=10, random_state=42)
rf_px.fit(X_px_train, y_train_ml)
rf_px_pred = rf_px.predict(X_px_test)

rfpx_mae = mean_absolute_error(y_test_ml, rf_px_pred)
rfpx_rmse = np.sqrt(mean_squared_error(y_test_ml, rf_px_pred))
rfpx_r2 = r2_score(y_test_ml, rf_px_pred)

print(f"✓ RF Pixel-Level Performance:")
print(f"  MAE:  {rfpx_mae:.3f} km²")
print(f"  RMSE: {rfpx_rmse:.3f} km²")
print(f"  R²:   {rfpx_r2:.3f}")

# -----------------------------------------------------------
# 5.3 XGBoost Regression
# -----------------------------------------------------------
print("\n" + "="*70)
print("5.3 XGBOOST REGRESSION")
print("="*70)

xgb = XGBRegressor(
    n_estimators=300, 
    learning_rate=0.05, 
    max_depth=6,
    subsample=0.8, 
    colsample_bytree=0.8, 
    random_state=42
)
xgb.fit(X_train, y_train_ml)
xgb_pred = xgb.predict(X_test)

xgb_mae = mean_absolute_error(y_test_ml, xgb_pred)
xgb_rmse = np.sqrt(mean_squared_error(y_test_ml, xgb_pred))
xgb_r2 = r2_score(y_test_ml, xgb_pred)

print(f"✓ XGBoost Performance:")
print(f"  MAE:  {xgb_mae:.3f} km²")
print(f"  RMSE: {xgb_rmse:.3f} km²")
print(f"  R²:   {xgb_r2:.3f}")

# -----------------------------------------------------------
# 5.4 SARIMA Model
# -----------------------------------------------------------
print("\n" + "="*70)
print("5.4 SARIMA (Seasonal ARIMA)")
print("="*70)

try:
    sarima_model = SARIMAX(
        y_train, 
        order=(1,1,1), 
        seasonal_order=(1,1,1,12),
        enforce_stationarity=False, 
        enforce_invertibility=False
    ).fit(disp=False)

    sarima_pred = sarima_model.forecast(len(y_test))

    sarima_mae = mean_absolute_error(y_test, sarima_pred)
    sarima_rmse = np.sqrt(mean_squared_error(y_test, sarima_pred))
    sarima_r2 = r2_score(y_test, sarima_pred)

    print(f"✓ SARIMA Performance:")
    print(f"  MAE:  {sarima_mae:.3f} km²")
    print(f"  RMSE: {sarima_rmse:.3f} km²")
    print(f"  R²:   {sarima_r2:.3f}")
except Exception as e:
    print(f"⚠ SARIMA model failed: {str(e)}")
    # Fallback predictions
    sarima_pred = np.full(len(y_test), y_train.mean())
    sarima_mae = mean_absolute_error(y_test, sarima_pred)
    sarima_rmse = np.sqrt(mean_squared_error(y_test, sarima_pred))
    sarima_r2 = r2_score(y_test, sarima_pred)
    print(f"✓ Using fallback (mean) predictions")

# ============================================================
# 6. MODEL COMPARISON & RESULTS
# ============================================================
print("\n" + "="*70)
print("6. MODEL COMPARISON & RESULTS")
print("="*70)

results = pd.DataFrame({
    "Model": ["Random Forest", "RF Pixel-Level", "XGBoost", "SARIMA"],
    "MAE (km²)": [rf_mae, rfpx_mae, xgb_mae, sarima_mae],
    "RMSE (km²)": [rf_rmse, rfpx_rmse, xgb_rmse, sarima_rmse],
    "R²": [rf_r2, rfpx_r2, xgb_r2, sarima_r2]
})

print("\n📊 Performance Comparison:")
print(results.to_string(index=False))

# Save results
results.to_csv(os.path.join(OUTPUT_DIR, "Model_Comparison_Results.csv"), index=False)
print(f"\n✓ Results saved to: Model_Comparison_Results.csv")

# Identify best model
best_idx = results["RMSE (km²)"].idxmin()
best_model = results.loc[best_idx, "Model"]
best_rmse = results.loc[best_idx, "RMSE (km²)"]

print(f"\n🏆 Best Performing Model: {best_model}")
print(f"   RMSE: {best_rmse:.3f} km²")

# Store best model object for later use
if best_model == "Random Forest":
    final_model = rf
    model_type = "RF"
elif best_model == "RF Pixel-Level":
    final_model = rf_px
    model_type = "RF-Pixel"
elif best_model == "XGBoost":
    final_model = xgb
    model_type = "XGBoost"
else:
    final_model = sarima_model
    model_type = "SARIMA"

# ============================================================
# 7. VISUALIZATION - FORECAST COMPARISON
# ============================================================
print("\n" + "="*70)
print("7. VISUALIZATION - FORECAST COMPARISON")
print("="*70)

fig, axes = plt.subplots(2, 1, figsize=(16, 12), sharex=True)

# Plot 1: Model predictions
axes[0].plot(df["Date"], df["Deforested_Area_km2"], 
            label="Observed", color="black", linewidth=2.5, marker='o', markersize=3)
axes[0].plot(X_test_index, rf_pred, "--", label="Random Forest", 
            linewidth=2, marker='s', markersize=4)
axes[0].plot(X_test_index, rf_px_pred, "--", label="RF Pixel-Level", 
            linewidth=2, marker='^', markersize=4)
axes[0].plot(X_test_index, xgb_pred, "--", label="XGBoost", 
            linewidth=2, marker='d', markersize=4)
axes[0].plot(X_test_index, sarima_pred, "--", label="SARIMA", 
            linewidth=2, marker='x', markersize=4)
axes[0].axvline(x=train_df['Date'].max(), color='orange', linestyle=':', 
               linewidth=2, alpha=0.7, label='Train/Test Split')
axes[0].set_title("Model Forecast Comparison - Matale District\n(Based on Cloud-Masked Data)", 
                 fontsize=14, fontweight='bold')
axes[0].set_ylabel("Deforested Area (km²)", fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Plot 2: Cloud coverage
if 'Cloud_Coverage' in df.columns:
    axes[1].bar(df["Date"], df["Cloud_Coverage"], color='lightblue', alpha=0.6, label='Cloud Coverage')
    axes[1].axhline(y=50, color='red', linestyle='--', linewidth=2, alpha=0.5, label='50% Threshold')
    axes[1].set_title("Cloud Coverage per Scene", fontsize=14, fontweight='bold')
    axes[1].set_ylabel("Cloud Coverage (%)", fontsize=12)
else:
    # Show residuals if no cloud data
    axes[1].plot(X_test_index, y_test.values - rf_pred, 'o-', label='RF Residuals', alpha=0.7)
    axes[1].plot(X_test_index, y_test.values - xgb_pred, 's-', label='XGBoost Residuals', alpha=0.7)
    axes[1].axhline(y=0, color='black', linestyle='--', linewidth=1)
    axes[1].set_title("Model Prediction Residuals", fontsize=14, fontweight='bold')
    axes[1].set_ylabel("Residual (km²)", fontsize=12)

axes[1].set_xlabel("Date", fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "Model_Forecast_Comparison.png"), dpi=200)
plt.show()

print("✓ Visualization saved to: Model_Forecast_Comparison.png")

# ============================================================
# 8. K-MEANS CLUSTERING ANALYSIS
# ============================================================
print("\n" + "="*70)
print("8. K-MEANS CLUSTERING ANALYSIS")
print("="*70)

# Clustering features
cluster_features = ["Deforested_Area_km2"]
if 'Cloud_Coverage' in df.columns:
    cluster_features.append("Cloud_Coverage")

X_cluster = StandardScaler().fit_transform(df[cluster_features])
kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)
df["Cluster"] = kmeans.fit_predict(X_cluster)

# Cluster statistics
cluster_stats = df.groupby('Cluster')['Deforested_Area_km2'].agg(['mean', 'min', 'max', 'count'])
print("\n✓ Cluster Statistics (Deforestation):")
print(cluster_stats)

if 'Cloud_Coverage' in df.columns:
    cloud_cluster_stats = df.groupby('Cluster')['Cloud_Coverage'].agg(['mean', 'min', 'max'])
    print("\n✓ Cluster Statistics (Cloud Coverage):")
    print(cloud_cluster_stats)

# Visualization
plt.figure(figsize=(14,6))
sns.scatterplot(x="Date", y="Deforested_Area_km2", hue="Cluster", 
               palette="Set2", data=df, s=60)
plt.title("Temporal Clustering of Deforestation Patterns (K=3)\nBased on Cloud-Masked Data", 
         fontsize=14, fontweight='bold')
plt.xlabel("Date", fontsize=12)
plt.ylabel("Deforested Area (km²)", fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(title="Cluster")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "KMeans_Temporal_Clusters.png"), dpi=200)
plt.show()

# Save clustered data
df.to_csv(os.path.join(OUTPUT_DIR, "Deforestation_with_Clusters.csv"), index=False)
print("\n✓ Clustered data saved to: Deforestation_with_Clusters.csv")

print("\n" + "="*70)
print("✅ MODEL TRAINING & COMPARISON COMPLETE!")
print("="*70)


DEFORESTATION PREDICTION MODELS - MATALE DISTRICT
Multi-Model Comparison: RF, RF-Pixel, XGBoost, SARIMA

2. DATA LOADING & CLOUD COVERAGE INTEGRATION
✓ Loaded monthly dataset: 218 rows
  Date range: 2013-05-26 → 2024-12-26
✓ Cloud coverage integrated


TypeError: unsupported format string passed to Series.__format__

In [ ]:
# ============================================================
# 9. LONG-TERM FORECASTING (2026-2030)
# ============================================================
print("\n" + "="*70)
print("9. LONG-TERM FORECASTING (2026-2030)")
print("="*70)

print(f"Using best model: {best_model} ({model_type})")

# ============================================================
# 9.1 Generate Future Dates
# ============================================================
last_date = df["Date"].max()
future_dates = pd.date_range(start=last_date + pd.offsets.MonthBegin(1),
                             end="2030-12-31", freq="MS")

future_df = pd.DataFrame({"Date": future_dates})

# Add temporal features
future_df["Month"] = future_df["Date"].dt.month
future_df["Year"] = future_df["Date"].dt.year
future_df["Quarter"] = future_df["Date"].dt.quarter

# Cloud coverage assumption: use training average for future
if "Cloud_Coverage" in df.columns and "Cloud_Coverage" in feature_cols:
    avg_cloud = df["Cloud_Coverage"].mean()
    future_df["Cloud_Coverage"] = avg_cloud
    print(f"✓ Using average cloud coverage for future: {avg_cloud:.1f}%")

# Valid pixels: use training average if available
if "Valid_Pixels" in df.columns and "Valid_Pixels" in feature_cols:
    avg_valid_pixels = df["Valid_Pixels"].mean()
    future_df["Valid_Pixels"] = avg_valid_pixels
    print(f"✓ Using average valid pixels: {avg_valid_pixels:,.0f}")

print(f"✓ Generated {len(future_df)} monthly predictions")

# ============================================================
# 9.2 Make Predictions Based on Best Model
# ============================================================
if model_type in ["RF", "XGBoost"]:
    # Tree-based models use temporal features
    future_preds = final_model.predict(future_df[feature_cols])

elif model_type == "RF-Pixel":
    # Pixel-level model: use average pixel features
    future_df["Pixel_Mean"] = df["Pixel_Mean"].mean()
    future_df["Pixel_Variance"] = df["Pixel_Variance"].mean()
    future_df["Pixel_Texture"] = df["Pixel_Texture"].mean()
    future_preds = final_model.predict(future_df[px_cols])

elif model_type == "SARIMA":
    # Time series model: direct forecast
    steps = len(future_df)
    future_preds = final_model.forecast(steps=steps)

future_df["Predicted_Deforested_Area_km2"] = future_preds

# ============================================================
# 9.3 Calculate Yearly Projections
# ============================================================
print("\n📊 Calculating yearly projections...")

yearly_forecast = future_df.groupby("Year")["Predicted_Deforested_Area_km2"].mean().reset_index()
yearly_forecast.columns = ["Year", "Projected_Area_km2"]

# Current baseline
baseline = df["Deforested_Area_km2"].iloc[-1]
current_year = df["Date"].iloc[-1].year

print(f"\n✓ Baseline ({current_year}): {baseline:.2f} km²")

# Calculate cumulative increase from baseline
cumulative_results = []

for _, row in yearly_forecast.iterrows():
    year = int(row["Year"])
    projected = row["Projected_Area_km2"]
    increase = projected - baseline
    increase_pct = (increase / baseline) * 100 if baseline > 0 else 0

    cumulative_results.append({
        "Year": year,
        "Projected_Area_km2": projected,
        "Cumulative_Increase_km2": increase,
        "Cumulative_Increase_Pct": increase_pct
    })

cumulative_df = pd.DataFrame(cumulative_results)

# ============================================================
# 9.4 Display & Save Results
# ============================================================
print("\n📈 Yearly Deforestation Projections:")
print("="*70)
print(cumulative_df.to_string(index=False))

# Highlight 2030 projection
projection_2030 = cumulative_df[cumulative_df["Year"] == 2030]
if len(projection_2030) > 0:
    final_2030 = projection_2030.iloc[0]
    print(f"\n🎯 2030 PROJECTION:")
    print(f"   Projected deforested area: {final_2030['Projected_Area_km2']:.2f} km²")
    print(f"   Increase from current: {final_2030['Cumulative_Increase_km2']:.2f} km² ({final_2030['Cumulative_Increase_Pct']:+.1f}%)")

# Save results
yearly_path = os.path.join(OUTPUT_DIR, "Yearly_Projections_2030.csv")
cumulative_df.to_csv(yearly_path, index=False)
print(f"\n✓ Yearly projections saved to: Yearly_Projections_2030.csv")

# Save full monthly forecast
monthly_path = os.path.join(OUTPUT_DIR, "Monthly_Forecast_2030.csv")
future_df.to_csv(monthly_path, index=False)
print(f"✓ Monthly forecast saved to: Monthly_Forecast_2030.csv")

# ============================================================
# 9.5 Visualization - Long-Term Forecast
# ============================================================
print("\n📊 Generating long-term forecast visualization...")

fig, axes = plt.subplots(2, 1, figsize=(16, 12), sharex=True)

# Plot 1: Historical + Forecast
axes[0].plot(df["Date"], df["Deforested_Area_km2"], 
            label="Historical Deforestation (Cloud-Masked)", 
            color="darkgreen", linewidth=3, marker='o', markersize=4)

axes[0].plot(future_df["Date"], future_df["Predicted_Deforested_Area_km2"], 
            label=f"{best_model} Long-term Forecast", 
            color="red", linewidth=2.5, linestyle='--', marker='s', markersize=3)

# Mark forecast start
axes[0].axvline(x=df["Date"].max(), color="orange", linestyle=":", 
               linewidth=2.5, alpha=0.8, label="Forecast Start")

# Mark 2030
axes[0].axvline(x=pd.to_datetime("2030-12-31"), color="purple", 
               linestyle=":", linewidth=2, alpha=0.6, label="2030 Target")

axes[0].set_title(f"Long-Term Deforestation Forecast - Matale District (2026-2030)\nModel: {best_model}", 
                 fontsize=16, fontweight='bold')
axes[0].set_ylabel("Deforested Area (km²)", fontsize=13)
axes[0].legend(fontsize=11, loc='upper left')
axes[0].grid(True, alpha=0.3, linestyle='--')

# Plot 2: Yearly projections as bar chart
axes[1].bar(cumulative_df["Year"], cumulative_df["Projected_Area_km2"], 
           color='steelblue', alpha=0.7, edgecolor='black')
axes[1].axhline(y=baseline, color='red', linestyle='--', 
               linewidth=2, alpha=0.7, label=f'Current Level ({baseline:.1f} km²)')
axes[1].set_title("Yearly Average Deforestation Projection", fontsize=14, fontweight='bold')
axes[1].set_xlabel("Year", fontsize=13)
axes[1].set_ylabel("Projected Area (km²)", fontsize=13)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "Long_Term_Forecast_2030.png"), dpi=200, bbox_inches='tight')
plt.show()

print("✓ Visualization saved to: Long_Term_Forecast_2030.png")

# ============================================================
# 10. SUMMARY & RECOMMENDATIONS
# ============================================================
print("\n" + "="*70)
print("10. SUMMARY & RECOMMENDATIONS")
print("="*70)

print(f"\n📁 All results saved to: {OUTPUT_DIR}")

print("\n📦 Generated Files:")
print("  • Model_Comparison_Results.csv - Performance metrics")
print("  • Model_Forecast_Comparison.png - Visual comparison")
print("  • KMeans_Temporal_Clusters.png - Clustering analysis")
print("  • Deforestation_with_Clusters.csv - Data with clusters")
print("  • Yearly_Projections_2030.csv - Yearly forecasts")
print("  • Monthly_Forecast_2030.csv - Monthly predictions")
print("  • Long_Term_Forecast_2030.png - Long-term visualization")

print("\n🌲 Conservation Applications:")
print("  • Plan forest conservation interventions")
print("  • Identify high-risk deforestation zones")
print("  • Set realistic reforestation targets")
print("  • Monitor progress against projected trends")
print("  • Allocate conservation resources effectively")

if 'Cloud_Coverage' in df.columns:
    print("\n⛅ Cloud Masking Impact:")
    print(f"  • Historical data filtered for <50% cloud coverage")
    print(f"  • Average cloud coverage: {df['Cloud_Coverage'].mean():.1f}%")
    print(f"  • Future predictions assume stable cloud patterns")
    print(f"  • Improved baseline accuracy through quality filtering")

print("\n⚠️ Important Considerations:")
print("  • Forecasts assume historical trends continue")
print("  • Policy changes or interventions may alter trajectory")
print("  • Cloud masking provides more accurate baseline")
print("  • Monitor actual vs. predicted for model validation")
print(f"  • Best performing model: {best_model} (RMSE: {best_rmse:.3f} km²)")

print("\n💡 Next Steps:")
print("  1. Validate predictions with field observations")
print("  2. Implement monitoring for high-risk zones")
print("  3. Develop intervention strategies for priority areas")
print("  4. Update model quarterly with new satellite data")
print("  5. Integrate with spatial risk analysis for targeted action")

print("\n" + "="*70)
print("✅ DEFORESTATION PREDICTION MODELING COMPLETE!")
print("="*70)